# Notebook para ajuste de erros no Cadastro Geral de Acompanhamento de Decisões

In [1]:
import os
import json
import pymssql

import pandas as pd
import numpy as np


from pprint import pprint
from langchain_openai import  AzureChatOpenAI, ChatOpenAI
from dotenv import load_dotenv
from sqlalchemy import (
    create_engine, Column, Integer, String, Float, Date, Boolean, Text, JSON
)
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.schema import DDL, CheckConstraint
from tools.prompt import generate_few_shot_ner_prompts
from tools.schema import (
    NERDecisao,
    Obrigacao
)

from datetime import timedelta


load_dotenv()


gpt4turbo = AzureChatOpenAI(
    deployment_name="gpt-4-turbo",
    model_name="gpt-4",
)

'''
gpt4turbo = ChatOpenAI(
    model="gpt-4-turbo",
    temperature=0.0
)
'''

extractor_gpt4turbo = gpt4turbo.with_structured_output(NERDecisao, include_raw=False, method="json_schema")
extractor_obrigacao_gpt4turbo = gpt4turbo.with_structured_output(
    Obrigacao, include_raw=False, method="json_schema")


def safe_int(value):
    if pd.isna(value):
        return None
    return int(value)

def get_connection(db: str = 'processo'):
    server = os.getenv("SQL_SERVER_HOST")
    user = os.getenv("SQL_SERVER_USER")
    password = os.getenv("SQL_SERVER_PASS")
    port = os.getenv("SQL_SERVER_PORT", "1433")  # default MSSQL port
    database = db

    return pymssql.connect(server, user, password, database, port)


c:\Users\05911205424\Documents\Dev\cgad\venv\Lib\site-packages\langchain_openai\chat_models\base.py:1927: UserWarning: Cannot use method='json_schema' with model gpt-4 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(
